In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from utils.black_scholes import raiz_ratio, d1_ratio, call_price_ratio
from utils.biseccion import bisec
import math
import scipy.stats as st
from utils.sample import Sample
import pandas as pd

def f(x):
    return raiz_ratio(c, ratio, r, x, T)


In [ ]:
## Para crear muestra

obj = Sample(ratio=[0.4, 1.6], T=[0.2,1.1], r=[0.02, 0.1], o=[0.01, 1])
obj.create('my_muestra', 10**5)

In [ ]:
#abro my muestra
obj = Sample()
x, y = obj.open('my_muestra')

In [ ]:
df = pd.DataFrame(x, columns=['c/k', 'ratio', 'r', 'T'])

In [ ]:
df['o'] = y
df

In [ ]:
def my_f(e):
    return raiz_ratio(e['c/k'], e['ratio'], e['r'], 0.01, e['T'])

def norm_1(e, o=0.01):
    return st.norm.cdf(d1_ratio(e['ratio'], e['r'], o, e['T']))

def norm_2(e, o=0.01):
    return st.norm.cdf(d1_ratio(e['ratio'], e['r'], o, e['T'])- o*math.sqrt(e['T']))

In [ ]:
def norm1p(e):
    return st.norm.cdf(d1_ratio(e['ratio'], e['r'], e['o'], e['T']))


def norm2p(e):
    return st.norm.cdf(d1_ratio(e['ratio'], e['r'], e['o'], e['T'])- e['o']*math.sqrt(e['T']))

In [ ]:
#tarda tiempo, agregar norm 1 y norm 2
df['f(0.01)'] = df.apply(my_f, axis=1)
df['norm1'] = df.apply(norm_1, axis=1) #es la normal de la volatilidad == 0.01
df['norm2'] = df.apply(norm_1, axis=1) #es la normal de la volatilidad == 0.01
df['norm1p'] = df.apply(norm1p, axis=1) #es la normal de la volatilidad implícita
df['norm2p'] = df.apply(norm2p, axis=1) #es la normal de la volatilidad implícita

In [ ]:
df.head(5)

In [ ]:
#casos en los que no puedo aplicar el método de bisección
casos_de_mandinga = df[df['f(0.01)'] > 0]
len(df[((df['norm1'] == 1) & (df['norm2'] == 1)) |
       ((df['norm1'] == 0) & (df['norm2'] == 0))])
len(casos_de_mandinga)

In [ ]:
# minimo y maximo de la normal calculada con la volatilidad implícita
# muy cercanos a 1
(casos_de_mandinga['norm1p'].min(), casos_de_mandinga['norm1p'].max()) 

In [ ]:
#Caso en particular con ratio
e = casos_de_mandinga.iloc[0]
o_posta = e['o']

def f(e, x):
    return raiz_ratio(e['c/k'], e['ratio'], e['r'], x, e['T'])

o = np.sort(np.append(np.linspace(0.01, 1, 100), np.array([o_posta])))
print(o)


In [ ]:
# grafico en funcion de f
fig = plt.figure(figsize=(8, 10), constrained_layout=True)
gs = gridspec.GridSpec(2, 3, figure=fig)
ax = fig.add_subplot(gs[0, :])
ax.plot(o, [f(e, i) for i in o], label = 'f(o)')
ax.plot([o_posta]*2, np.linspace(0,0.1,2), 'r--', label = 'volatilidad implicita')
ax.set_xlabel('volatilidad')
ax.set_ylabel('g(n)')
#ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
plt.grid()



In [ ]:
#grafico en funcion de norm1
fig = plt.figure(figsize=(8, 10), constrained_layout=True)
gs = gridspec.GridSpec(2, 3, figure=fig)
ax = fig.add_subplot(gs[0, :])
aux = np.linspace(0.01, round(o_posta+0.005, 3), 101)
ax.plot(aux, [norm_1(e, o=i) for i in aux], label = 'norm1(o)')
ax.plot([o_posta]*2, np.linspace(0,1,2), 'r--', label = 'volatilidad implicita')
ax.set_xlabel('volatilidad')
ax.set_ylabel('norm1(o)')
#ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
plt.grid()


In [ ]:
#grafico en funcion de norm2
fig = plt.figure(figsize=(8, 10), constrained_layout=True)
gs = gridspec.GridSpec(2, 3, figure=fig)
ax = fig.add_subplot(gs[0, :])
ax.plot(aux, [norm_2(e, o=i) for i in aux], label = 'norm2(o)')
ax.plot([o_posta]*2, np.linspace(0,1,2), 'r--', label = 'volatilidad implicita')
ax.set_xlabel('volatilidad')
ax.set_ylabel('norm2(o)')
#ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
plt.grid()